In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import docker
import time
import concurrent.futures
from datetime import datetime
import copy
import math
from collections import defaultdict
import pprint
from collections import namedtuple
from datetime import datetime
import os
import ctypes as ct
import multiprocessing
import rapl

In [10]:
dir(rapl)

['JOULES',
 'MILLIJOULES',
 'RAPLDifference',
 'RAPLDomain',
 'RAPLMonitor',
 'RAPLSample',
 'UJOULES',
 'WATT_HOURS',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'datetime',
 'os',
 'rapl',
 're']

In [10]:
# Discover the hardware architecture.
avail_cores = os.cpu_count()
print(f"Available cores: {avail_cores}")

Available cores: 48


In [ ]:
# Run docker-activity to monitor power consumption and runtime of the containers.

# Initialize Docker client
client = docker.from_env()
activity_container = client.containers.run(
    image="jdrouet/docker-activity",
    command=["stdout"],
    volumes={
        "/sys/class/powercap": {"bind": "/sys/class/powercap", "mode": "ro"},
        "/var/run/docker.sock": {"bind": "/var/run/docker.sock", "mode": "rw"},
    },
    privileged=True,
    detach=True,
    auto_remove=True
)
for log in activity_container.logs(stream=True, follow=True):
    print(log.decode().strip())

In [ ]:
s1 = rapl.RAPLMonitor.sample()
# some code to measure
s2 = rapl.RAPLMonitor.sample()
print((s2 - s1).energy("package-0", "core", rapl.UJOULES))
rapl.R

# Read total energy at T0
# Run the isolated benchmarks
# Read total CPU time of each container during benchmark execution
# Read total energy at T1 after benchmarks are finished


275


In [26]:
# Run isolated benchmarks in a Docker container on the different cores.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels={"test": next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)}
    )
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    # Ensure start_time is set even if the container does not reach 'running'
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
            
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    isolated_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=50G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # List of dictionaries to hold task information.
    tasks = [
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=800000000000","run"], "cpuset_cpus": "1"},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "2"},
        {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=50G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "3"}
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Isolator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    for future in futures:
        future.result()
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All isolated benchmarks completed.")
    # Clean up Docker client
    client.close()  

Container started for Disk benchmark (prepare).
Container for benchmark ['sysbench', '--test=cpu', '--num-threads=8', '--cpu-max-prime=800000000000', 'run'] started successfully.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=1M', '--memory-total-size=10G', '--threads=1', 'run'] started successfully.
Container lifetime: 0.164251 seconds
Container on CPU 1 completed and removed.
Container lifetime: 9.617084 seconds
Container on CPU 2 completed and removed.
Container for benchmark ['sysbench', '--test=fileio', '--file-total-size=50G', '--file-test-mode=rndrw', '--init-rng=on', '--max-time=300', '--max-requests=0', 'run'] started successfully.
Container lifetime: 0.468979 seconds
Container on CPU 3 completed and removed.
Disk benchmark completed.
All isolated benchmarks completed.


In [24]:
# Uses linpack | Run isolated benchmarks in a Docker container on the different cores.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    label = task.get("label")
    if label is None:
        label = next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels = {"test": label}
    )
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    # Ensure start_time is set even if the container does not reach 'running'
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
            
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    isolated_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=50G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # List of dictionaries to hold task information.
    tasks = [
        {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "1", "label":"cpu"},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "2"},
        {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=50G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "3"}
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Isolator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    for future in futures:
        future.result()
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All isolated benchmarks completed.")
    # Clean up Docker client
    client.close()  

Container started for Disk benchmark (prepare).


KeyboardInterrupt: 

In [1]:
# Access the watched benchmark containers for runtime and power consumption.
print("Container names and IDs:")
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

Container names and IDs:


NameError: name 'isolated_benchmarking_results' is not defined

In [ ]:
# Run co-located benchmarks in a Docker container on the same core.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels={"test": next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)}
    )
    
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'colocated_runtime': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=300G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "0"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "24"}
        ]},
        {"MemFileIO": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=150G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "25"}
        ]},
        {"FileIOCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "2"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "26"}
        ]},
        {"CpuCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "3"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "27"},
        ]},
        {"MemMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "4"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "28"},
        ]},
        {"FileIOFileIO": [
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "5"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "29"}
        ]}
    ]
    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 

Container started for Disk benchmark (prepare).
Running colocated tasks for: CpuMem
Container for benchmark ['sysbench', '--test=cpu', '--num-threads=8', '--cpu-max-prime=40000000000', 'run'] started successfully.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=8M', '--memory-total-size=150G', '--threads=1', 'run'] started successfully.
Container lifetime: 0.176359 seconds
Container on CPU 0 completed and removed.
Container lifetime: 6.177636 seconds
Container on CPU 24 completed and removed.
Completed colocated tasks for: CpuMem
Running colocated tasks for: MemFileIO
Container for benchmark ['sysbench', '--test=fileio', '--file-total-size=150G', '--file-test-mode=rndrw', '--init-rng=on', '--max-time=300', '--max-requests=0', 'run'] started successfully.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=8M', '--memory-total-size=150G', '--threads=1', 'run'] started successfully.
Container lifetime: 0.47887 seconds
Container on CPU 25 

In [4]:
# Using linpack | Run co-located benchmarks in a Docker container on the same core.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    label = task.get("label")
    if label is None:
        label = next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels={"test": label}
    )
    
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'colocated_runtime': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=300G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
            {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "1", "label": "cpu"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "24"}
        ]},
        {"MemFileIO": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=150G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "25"}
        ]},
        {"FileIOCpu": [
            {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "1", "label": "cpu"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "26"}
        ]},
        {"CpuCpu": [
            {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "1", "label": "cpu"},
            {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "1", "label": "cpu"},
        ]},
        {"MemMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "4"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "28"},
        ]},
        {"FileIOFileIO": [
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "5"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "29"}
        ]}
    ]
    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 

Container started for Disk benchmark (prepare).
Running colocated tasks for: CpuMem
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=8M', '--memory-total-size=150G', '--threads=1', 'run'] started successfully.
Container for benchmark ['linpack'] started successfully.
Container lifetime: 6.161869 seconds
Container on CPU 24 completed and removed.
Container lifetime: 10.247913 seconds
Container on CPU 1 completed and removed.
Completed colocated tasks for: CpuMem
Running colocated tasks for: MemFileIO
Container for benchmark ['sysbench', '--test=fileio', '--file-total-size=150G', '--file-test-mode=rndrw', '--init-rng=on', '--max-time=300', '--max-requests=0', 'run'] started successfully.
Container lifetime: 0.386965 seconds
Container on CPU 25 completed and removed.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=8M', '--memory-total-size=150G', '--threads=1', 'run'] started successfully.
Container lifetime: 7.176097 seconds
Container 

In [11]:
# Access the watched benchmark containers for runtime and power consumption.
coloc_benchmarking_results_copy = copy.deepcopy(coloc_benchmarking_results)

print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")
# print(benchmarking_results)

Container names and IDs:
eloquent_sutherland: {'coloc_pair': 'CpuMem', 'workload': 'memory', 'id': '30a982abcd6211ec01473caa5fdedc5340d3a0bb6db5dbe1e73ebed82471d017', 'colocated_runtime': 6.161869, 'isolated_runtime': 0.638697, 'isolated_power_consumption': 'not yet computed'}
relaxed_gagarin: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': '2498536fe8d1f2b24834ed86b5aecadb278a3bf7493326ced6bc083d241b3bb1', 'colocated_runtime': 10.247913, 'isolated_runtime': 10.369488, 'isolated_power_consumption': 'not yet computed'}
nifty_wilson: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': '3e225ecf6ec7a378180c8214d3e9c8ab17fdeba10907692420282ab7ce7c952c', 'colocated_runtime': 0.386965, 'isolated_runtime': 0.168125, 'isolated_power_consumption': 'not yet computed'}
dreamy_lamarr: {'coloc_pair': 'MemFileIO', 'workload': 'memory', 'id': '0dda809d09b83bc6891c779c43887eb6c5ed3da6bfa9562b447d490c54b98e5e', 'colocated_runtime': 7.176097, 'isolated_runtime': 0.638697, 'isolated_power_consumpti

In [14]:
def calc_average_slowdown(slowdown_1, slowdown_2):
    average_slowdown = (slowdown_1 + slowdown_2) / 2
    return average_slowdown

def calc_slowdown_factor(isolated_runtime_1, isolated_runtime_2, coloc_runtime_1, coloc_runtime_2):
    slowdown_1 = isolated_runtime_1 / coloc_runtime_1
    slowdown_2 = isolated_runtime_2 / coloc_runtime_2
    average_slowdown = calc_average_slowdown(slowdown_1, slowdown_2)
    return average_slowdown

def calc_affinity_score(isolated_runtime_1, coloc_runtime_1, isolated_runtime_2, coloc_runtime_2):
    affinity_score = (isolated_runtime_1 + isolated_runtime_2) / (coloc_runtime_1 + coloc_runtime_2)
    return min(1, affinity_score)
    
for name, result in coloc_benchmarking_results.items():
    coloc_workload = result.get('workload')
    # Find the isolated runtime for the same workload
    isolated_runtime = ''
    for iso_name, iso_result in isolated_benchmarking_results.items():
        if iso_result.get('workload') == coloc_workload and iso_result.get('coloc_pair') is None:
            isolated_runtime = iso_result.get('life_time')
            break
    result['isolated_runtime'] = isolated_runtime
    result['isolated_power_consumption'] = 'not yet computed'
    print(f"Colocated benchmark: {coloc_workload}, {result.get('coloc_pair')}, Isolated runtime: {isolated_runtime}")

Colocated benchmark: memory, CpuMem, Isolated runtime: 0.638697
Colocated benchmark: cpu, CpuMem, Isolated runtime: 10.369488
Colocated benchmark: fileio, MemFileIO, Isolated runtime: 0.168125
Colocated benchmark: memory, MemFileIO, Isolated runtime: 0.638697
Colocated benchmark: fileio, FileIOCpu, Isolated runtime: 0.168125
Colocated benchmark: cpu, FileIOCpu, Isolated runtime: 10.369488
Colocated benchmark: cpu, CpuCpu, Isolated runtime: 10.369488
Colocated benchmark: cpu, CpuCpu, Isolated runtime: 10.369488
Colocated benchmark: memory, MemMem, Isolated runtime: 0.638697
Colocated benchmark: memory, MemMem, Isolated runtime: 0.638697
Colocated benchmark: fileio, FileIOFileIO, Isolated runtime: 0.168125
Colocated benchmark: fileio, FileIOFileIO, Isolated runtime: 0.168125


In [22]:
from collections import defaultdict
import pprint

# Transform benchmarking dict into a summary
coloc_summary = defaultdict(dict)

for name, result in coloc_benchmarking_results.items():
    pair = result['coloc_pair']
    workload = result.get('workload', 'unknown')
    if 'workload_1' not in coloc_summary[pair]:
        coloc_summary[pair]['workload_1'] = workload
        coloc_summary[pair]['colocated_runtime_1'] = result.get('life_time')
        coloc_summary[pair]['isolated_runtime_1'] = result.get('isolated_runtime')
        coloc_summary[pair]['power_consumption_1'] = result.get('isolated_power_consumption')
    else:
        coloc_summary[pair]['workload_2'] = workload
        coloc_summary[pair]['colocated_runtime_2'] = result.get('life_time')
        coloc_summary[pair]['isolated_runtime_2'] = result.get('isolated_runtime')
        coloc_summary[pair]['power_consumption_2'] = result.get('isolated_power_consumption')

# Now calculate and add average_slowdown and affinity_score for each coloc pair
for pair, summary in coloc_summary.items():
    try:
        iso1 = float(summary['isolated_runtime_1'])
        iso2 = float(summary['isolated_runtime_2'])
        coloc1 = float(summary['colocated_runtime_1'])
        coloc2 = float(summary['colocated_runtime_2'])
        summary['average_slowdown'] = calc_slowdown_factor(iso1, iso2, coloc1, coloc2)
        summary['affinity_score'] = calc_affinity_score(iso1, coloc1, iso2, coloc2)
    except Exception as e:
        summary['average_slowdown'] = None
        summary['affinity_score'] = None
        print(f"Could not calculate for pair {pair}: {e}")

coloc_summary = dict(coloc_summary)
pprint.pprint(coloc_summary)

Could not calculate for pair CpuMem: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair MemFileIO: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair FileIOCpu: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair CpuCpu: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair MemMem: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair FileIOFileIO: float() argument must be a string or a real number, not 'NoneType'
{'CpuCpu': {'affinity_score': None,
            'average_slowdown': None,
            'colocated_runtime_1': None,
            'colocated_runtime_2': None,
            'isolated_runtime_1': 10.369488,
            'isolated_runtime_2': 10.369488,
            'power_consumption_1': 'not yet computed',
            'power_consumption_2': 'not yet computed',
        